This is the scraper I used.  It connects to the yelp API, taking in my unique api key, the term (ie 'coffee', 'noodles', 'burgers'), location, and radius (in meters).  Data is saved as a json files, with each offset saved as a new file.  

In [1]:
# Imports for Yelp
import requests
from getAPIKey import get_api_key

# Imports for saving 
import json

In [2]:
class BusinessData:
# Create functions to call on and save json data from Yelp API

    def __init__(self, apikey, term, location, radius):
    # initialize api key, endpoint (yelp's business search in this case), and parameters.
        self.apikey = apikey
        self.term = term
        self.location = location
        self.radius = radius
        self.offset = 0
        self.limit = 50
        self.business_endpoint = 'https://api.yelp.com/v3/businesses/search'
        self.header = {'Authorization': 'Bearer {}'.format(api_key)}

    def make_request(self):
    # Make request to Yelp API endpoint.  If response code is not 200, stop request.
    # Returns 1) data in json format and 2) total number of businesses
        response = requests.get(url = self.business_endpoint, params = self.get_params(), headers = self.header)
        if response.status_code != 200:
            print('Bad request.  Status code {}'.format(response.status_code))
        else:
            return response.json(), response.json()['total']

    def get_params(self):
    # Organize parameters into dictionary format
        PARAMS = {'term': self.term, 
                'location': self.location, 
                'radius': self.radius,
                'offset': self.offset,
                'limit': self.limit}
        return PARAMS
    
    def save_json(self, data, filename):
    # Save json data.
    # Need to find a way to append to existing json file instead of making new one each offset
        with open('{}.json'.format(filename), "w") as json_file:
            json.dump(data['businesses'], json_file, indent=4)

    def change_offset(self):
        self.offset += self.limit

In [4]:
api_key = get_api_key()

city = BusinessData(api_key, 'kbbq', 'Los Angeles', 10000)

In [5]:
city_data, city_total = city.make_request()

In [6]:
# Save all json data into separate files based on total number of businesses found.
for i in range(city_total//50+1):
    if i == 0:
        city.save_json(city_data, filename = 'kbbq{}'.format(i))

    else:
        city.change_offset()
        city_newdata, _ = city.make_request()
        city.save_json(city_newdata, filename = 'kbbq{}'.format(i))